# 1. get data

In [9]:
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime,date
import json

with open('krx_total_list.txt','r') as f:
    tickers = json.loads(f.read())
    
today = date.today()
start = datetime(2008, 1, 1)
end = datetime(today.year, today.month, today.day)
def get_data(n1, n2):
    cnt = 0
    print('processing',end='')
    for i,v in tickers.items():
        cnt+=1
        if cnt > n1 and '마켓' in v:
            ticker = i+'.'+v['마켓']
            filename = './data10y_2/'+i
            try:
                history = web.DataReader(ticker,'yahoo',start, end)
                history.to_pickle(filename)
                print('.',end='')
            except Exception as e:
                print(cnt,'failed',ticker,v['회사명'],e)
                pass
        if cnt == n2 :
            break
    print('download finished.')

In [10]:
get_data(0,10)

processing..........download finished.


In [11]:
get_data(10,1000)

processing.................................................................................................................................................................................................204 failed 099340.KS 하나니켈1호 'Date'
205 failed 099350.KS 하나니켈2호 'Date'
..............220 failed 096300.KS 베트남개발1 'Date'
......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [13]:
get_data(1000,2085)

processing..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

# 2. prediction(multi-processing)

In [1]:
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime,date
import json

with open('krx_total_list.txt','r') as f:
    tickers = json.loads(f.read())

In [8]:
# 10 year data

from fbprophet import Prophet
from multiprocessing import Pool
from time import time as t
start = t()

n1 = 1000
n2 = 2085
tickerlist = list(tickers.keys())
num_list = list(range(n1,n2))
print('make prediction : {} ~ {}'.format(n1,n2))
def sub_func(i):
    ticker = tickerlist[i]
    try:
        data = pd.read_pickle('./data10y_2/{}'.format(ticker))
        m = Prophet(daily_seasonality=True)
        df = pd.DataFrame({'ds':data.index, 'y':data['Close']})
        m.fit(df);
        future = m.make_future_dataframe(periods=365)
        forecast = m.predict(future)
        
        new_df = pd.DataFrame({'ds':forecast.ds,'yhat':forecast.yhat.round(2)}).set_index('ds')
        new_df = new_df.join(pd.DataFrame({'y':df.y}))
        new_df.y = new_df.y.fillna(0)
        new_df.to_pickle('./data10y_predict_pickle_2/{}'.format(ticker))
        print(i,ticker,'.',end='')
    except Exception as e:
        print(i,ticker,'failed',e)
        pass

pool = Pool(processes=2)
pool.map(sub_func, num_list)
print('prediction finished.')
    
print('time:', t()-start)


make prediction : 1000 ~ 2085


/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.p

1000 232140 .1001 191410 .1002 223310 .1003 145020 .1004 222980 .1005 206560 .1006 013310 .1007 217730 failed Dataframe has less than 2 non-NaN rows.


/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.p

/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.p

...

/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.p

/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.p

2046 011320 .2047 018120 .2048 013720 .2049 009620 .2050 005670 .2051 007330 .2052 006140 .2053 019010 .2054 009780 .2055 017680 .2056 008800 .2057 020400 .2058 014200 .2059 005710 .2060 016600 .2061 004590 .2062 016670 .2063 000440 .2064 014970 .2065 011040 .2066 008370 .2067 007390 .2068 002290 .2069 019540 .2070 007770 .2071 017890 .2072 003310 .2073 019660 .2074 017510 .2075 021080 .2076 008830 .2077 002230 .2078 009730 .2079 016920 .2080 013030 .2081 019550 .2082 019590 failed Dataframe has less than 2 non-NaN rows.


/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/jw/anaconda3/envs/prophet/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


prediction finished.
time: 7379.265493154526


In [18]:
check = pd.read_pickle('./data10y_predict_pickle_2/003220')
check.iloc[-370:-330]

,yhat,y
ds,,
2018-12-21,18515.66,16150.0
2018-12-24,18564.14,16000.0
2018-12-26,18642.09,15600.0
2018-12-27,18682.09,15750.0
2018-12-28,18732.46,16150.0
2018-12-29,18862.72,0.0
2018-12-30,18898.63,0.0
2018-12-31,18803.39,0.0
2019-01-01,18850.12,0.0


In [19]:
check = pd.read_pickle('./data10y_predict_pickle/003220')
check.iloc[-370:-330]

,yhat,y
ds,,
2018-11-28,18715.21,16550
2018-11-29,18723.74,16600
2018-11-30,18744.28,16700
2018-12-03,18728.74,16850
2018-12-04,18746.03,16850
2018-12-05,18770.69,null
2018-12-06,18789.02,null
2018-12-07,18819.56,null
2018-12-08,18916.60,null
